In [ ]:
#pip install pyspark

In [1]:
##import libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
#start a spark session
spark=SparkSession.builder\
      .appName("xyz_log_analysis").getOrCreate()


23/09/29 17:01:46 WARN Utils: Your hostname, dilips-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
23/09/29 17:01:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/29 17:01:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
##for poc set schemainference True
spark.sql("set spark.sql.streaming.schemaInference=True")


DataFrame[key: string, value: string]

In [4]:
#check schema of the file 

transactions_df = spark.read.json("/Users/dilips/Downloads/PSI_DE_Test_Instructions/input_data_generator/input_data/starter/transactions/d=2023-03-27")
transactions_df.printSchema()
#transactions_df.show(24)

root
 |-- basket: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- price: long (nullable = true)
 |    |    |-- product_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- date_of_purchase: string (nullable = true)



In [5]:
##create schema object for JSON
basket_schema = ArrayType(
    StructType([
        StructField("price", LongType(), nullable=True),
        StructField("product_id", StringType(), nullable=True)
    ]),
    containsNull=True
)

# Define the overall schema for your data
schema = StructType([
    StructField("basket", basket_schema, nullable=True),
    StructField("customer_id", StringType(), nullable=True),
    StructField("date_of_purchase", StringType(), nullable=True)
])


In [6]:
##batch - read customer and product
customoer_df = spark.read.format("csv").option("header","true").load("/Users/dilips/Downloads/PSI_DE_Test_Instructions/input_data_generator/input_data/starter/customers.csv")
product_df = spark.read.format("csv").option("header","true").load("/Users/dilips/Downloads/PSI_DE_Test_Instructions/input_data_generator/input_data/starter/products.csv")


In [7]:
##Since transaction is ever increasing data source create a stream reader

streaming_df = (
    spark.readStream.format("json")\
    .option("inferSchema", "true")\
    .option("maxFilesPerTrigger", 1)
    .option("header","true")
    .load("/Users/dilips/Downloads/PSI_DE_Test_Instructions/input_data_generator/input_data/starter/transactions/")
    .select(col("customer_id"), col("date_of_purchase"), explode("basket").alias("basket_item"))
    .select(col("customer_id"), col("date_of_purchase").cast("timestamp"),col("basket_item.price"), col("basket_item.product_id"))
)

#str_df = streaming_df.select(col("customer_id"), col("date_of_purchase"),col("basket_item.price"), col("basket_item.product_id"))


In [8]:
streaming_df.isStreaming

True

23/09/29 17:02:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
##Calculate purchase_count and purchase_frequency_pd columns over windows of 180 days

windowed_df = streaming_df \
    .groupBy("customer_id", "product_id", window("date_of_purchase", "180 days")) \
    .agg(
        avg("price").alias("avg_sales_amount"),
        min("date_of_purchase").alias("first_purchase_timestamp"),
        count("price").alias("purchase_count")
    )


windowed_df = windowed_df.withColumn(
    "purchase_frequency_pd",
    when(
        col("first_purchase_timestamp").isNotNull(),
        col("purchase_count") / datediff(windowed_df["window.end"], col("first_purchase_timestamp"))
    ).otherwise(None)
)



In [13]:
##drop unwanted columns
windowed_df= windowed_df.join(customoer_df, "customer_id", "left").join(product_df, "product_id","left").drop("product_description")\
                        .drop(*['window','avg_sales_amount','first_purchase_timestamp'])

In [ ]:
# streaming_df.writeStream.option("checkpointLocation", "/Users/dilips/Downloads/PSI_DE_Test_Instructions/input_data_generator/checkpoints/") \
#                .toTable("rawTransactionTable")


##print result in console
query = windowed_df.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination(10)

               

23/09/29 17:27:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/bl/fnn1wmns17xcc_jvgd66ng2r0000gn/T/temporary-303e1929-054e-44ee-a049-c0d91496cb3e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/09/29 17:27:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----------+-----------+--------------+---------------------+-------------+----------------+
|product_id|customer_id|purchase_count|purchase_frequency_pd|loyalty_score|product_category|
+----------+-----------+--------------+---------------------+-------------+----------------+
|       P27|        C23|             1| 0.006024096385542169|           10|       fruit_veg|
|       P50|        C33|             1| 0.006024096385542169|            5|            food|
|       P62|        C88|             1| 0.006024096385542169|            1|             bws|
|       P24|        C22|             1| 0.005988023952095809|            1|       fruit_veg|
|       P06|        C20|             1| 0.006024096385542169|            2|           house|
|       P38|        C46|             1| 0.005988023952095809|            5|          sweets|
|       P22|        C22|             1| 0.005988023952095809|     

-------------------------------------------
Batch: 1
-------------------------------------------
+----------+-----------+--------------+---------------------+-------------+----------------+
|product_id|customer_id|purchase_count|purchase_frequency_pd|loyalty_score|product_category|
+----------+-----------+--------------+---------------------+-------------+----------------+
|       P51|        C71|             2| 0.011494252873563218|            8|            food|
|       P42|        C71|             1| 0.005747126436781609|            8|            food|
|       P39|        C48|             1| 0.005747126436781609|            4|          sweets|
|       P37|       C103|             1| 0.005714285714285714|            7|          sweets|
|       P44|        C33|             1| 0.005714285714285714|            5|            food|
|       P55|        C62|             1| 0.005714285714285714|            6|            food|
|       P06|        C53|             1| 0.005714285714285714|     

-------------------------------------------
Batch: 2
-------------------------------------------
+----------+-----------+--------------+---------------------+-------------+----------------+
|product_id|customer_id|purchase_count|purchase_frequency_pd|loyalty_score|product_category|
+----------+-----------+--------------+---------------------+-------------+----------------+
|       P55|        C28|             1| 0.005847953216374269|            3|            food|
|       P30|        C12|             1| 0.005813953488372093|            8|       fruit_veg|
|       P37|        C20|             1| 0.005813953488372093|            2|          sweets|
|       P29|        C77|             1| 0.005847953216374269|            9|       fruit_veg|
|       P23|        C66|             1| 0.005813953488372093|           10|       fruit_veg|
|       P23|        C12|             1| 0.005813953488372093|            8|       fruit_veg|
|       P30|        C66|             1| 0.005813953488372093|     

-------------------------------------------
Batch: 3
-------------------------------------------
+----------+-----------+--------------+---------------------+-------------+----------------+
|product_id|customer_id|purchase_count|purchase_frequency_pd|loyalty_score|product_category|
+----------+-----------+--------------+---------------------+-------------+----------------+
|       P29|       C120|             1| 0.006060606060606061|            9|       fruit_veg|
|       P07|        C97|             1| 0.006060606060606061|            9|           house|
|       P62|        C88|             2| 0.012048192771084338|            1|             bws|
|       P23|       C120|             1| 0.006060606060606061|            9|       fruit_veg|
|       P44|        C94|             1| 0.006097560975609756|           10|            food|
|       P11|        C38|             1| 0.006060606060606061|            7|           house|
|       P64|        C32|             1| 0.006097560975609756|     

...

-------------------------------------------
Batch: 40
-------------------------------------------
+----------+-----------+--------------+---------------------+-------------+----------------+
|product_id|customer_id|purchase_count|purchase_frequency_pd|loyalty_score|product_category|
+----------+-----------+--------------+---------------------+-------------+----------------+
|       P41|        C74|             1| 0.008264462809917356|            5|            food|
|       P54|        C74|             1| 0.008264462809917356|            5|            food|
|       P43|        C25|             2| 0.011976047904191617|            9|            food|
|       P64|        C58|             2| 0.012903225806451613|            6|             bws|
|       P32|       C123|             1| 0.008333333333333333|            3|       fruit_veg|
|       P29|        C83|             2| 0.012195121951219513|            8|       fruit_veg|
|       P23|       C123|             2| 0.013333333333333334|    

-------------------------------------------
Batch: 41
-------------------------------------------
+----------+-----------+--------------+---------------------+-------------+----------------+
|product_id|customer_id|purchase_count|purchase_frequency_pd|loyalty_score|product_category|
+----------+-----------+--------------+---------------------+-------------+----------------+
|       P42|        C50|             1|            0.0078125|            8|            food|
|       P26|        C46|             1|            0.0078125|            5|       fruit_veg|
|       P63|        C83|             1| 0.007751937984496124|            8|             bws|
|       P28|       C111|             2| 0.013513513513513514|            1|       fruit_veg|
|       P28|       C104|             1|            0.0078125|            7|       fruit_veg|
|       P17|       C131|             1|            0.0078125|            4|         clothes|
|       P24|         C6|             1| 0.007751937984496124|    

-------------------------------------------
Batch: 42
-------------------------------------------
+----------+-----------+--------------+---------------------+-------------+----------------+
|product_id|customer_id|purchase_count|purchase_frequency_pd|loyalty_score|product_category|
+----------+-----------+--------------+---------------------+-------------+----------------+
|       P37|         C2|             5|  0.03184713375796178|            4|          sweets|
|       P26|        C46|             2| 0.015503875968992248|            5|       fruit_veg|
|       P32|         C3|             1| 0.007751937984496124|            8|       fruit_veg|
|       P09|        C30|             1| 0.007692307692307693|            4|           house|
|       P48|        C74|             1| 0.007692307692307693|            5|            food|
|       P30|        C73|             1| 0.007692307692307693|            2|       fruit_veg|
|       P28|        C73|             2| 0.012578616352201259|    

-------------------------------------------
Batch: 43
-------------------------------------------
+----------+-----------+--------------+---------------------+-------------+----------------+
|product_id|customer_id|purchase_count|purchase_frequency_pd|loyalty_score|product_category|
+----------+-----------+--------------+---------------------+-------------+----------------+
|       P05|        C26|             1| 0.006802721088435374|            1|           house|
|       P45|        C50|             1| 0.006802721088435374|            8|            food|
|       P61|        C53|             1| 0.006802721088435374|            4|             bws|
|       P38|       C132|             1| 0.006756756756756757|            7|          sweets|
|       P39|         C1|             1| 0.006756756756756757|            7|          sweets|
|       P58|       C116|             1| 0.006802721088435374|            5|            food|
|       P64|        C58|             3|  0.01935483870967742|    

-------------------------------------------
Batch: 44
-------------------------------------------
+----------+-----------+--------------+---------------------+-------------+----------------+
|product_id|customer_id|purchase_count|purchase_frequency_pd|loyalty_score|product_category|
+----------+-----------+--------------+---------------------+-------------+----------------+
|       P27|        C23|             5|  0.02976190476190476|           10|       fruit_veg|
|       P37|       C103|             2| 0.011428571428571429|            7|          sweets|
|       P26|        C47|             2| 0.011764705882352941|            2|       fruit_veg|
|       P03|        C28|             1| 0.007194244604316547|            3|           house|
|       P37|        C20|             4| 0.023255813953488372|            2|          sweets|
|       P48|        C33|             3| 0.017241379310344827|            5|            food|
|       P48|        C74|             2| 0.014388489208633094|    

In [15]:
query.stop()

23/09/29 17:28:13 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 45, writer: ConsoleWriter[numRows=20, truncate=true]] is aborting.
23/09/29 17:28:13 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 45, writer: ConsoleWriter[numRows=20, truncate=true]] aborted.
23/09/29 17:28:13 WARN Shell: Interrupted while joining on: Thread[#341545,Thread-340888,5,main]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait0(Native Method)
	at java.base/java.lang.Object.wait(Object.java:366)
	at java.base/java.lang.Thread.join(Thread.java:2115)
	at java.base/java.lang.Thread.join(Thread.java:2191)
	at org.apache.hadoop.util.Shell.joinThread(Shell.java:1042)
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1002)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.ha